In [1]:
!pip install econml
!pip install pyreadr

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.metrics import mean_squared_error
import sys
from econml.metalearners import TLearner, SLearner, XLearner, DomainAdaptationLearner
import numpy as np
from numpy.random import binomial, multivariate_normal, normal, uniform
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
import matplotlib.pyplot as plt
import pyreadr
%matplotlib inline
from econml.dr import DRLearner
from functools import total_ordering

# Enter Dataset Name : 'IHDP', 'jobs', 'cattaneo'
dataset = 'IHDP'

/Users/pnans/anaconda3/envs/cfrnet/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
if dataset == 'IHDP':
    def ihdp_data_prep():
        data = pd.read_csv('data/ihdp_npci_1.csv', header=None)
        col =  ["treatment", "y_factual", "y_cfactual", "mu0", "mu1" ,]
        for i in range(1,26):
            col.append(str(i))
        data.columns = col
        true_ate = np.mean(data['mu1']-data['mu0'])
        ihdp_data = pyreadr.read_r('data/ihdp.RData')['ihdp']
        ihdp_data = ihdp_data[(ihdp_data['treat'] != 1) | (ihdp_data['momwhite'] != 0)].reset_index(drop=True)
        ihdp_data = ihdp_data.drop(['momwhite', 'momblack', 'momhisp'], axis=1)
        ihdp_data_compressed = ihdp_data.copy()
        cols_to_norm = ['bw','b.head','preterm','birth.o','nnhealth','momage']
        ihdp_data_compressed[cols_to_norm] = ihdp_data_compressed[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.std()))
        ihdp_data_compressed.columns = ['treatment','birth-weight','head-circumference','pre-term','birth-order','neonatal','age','sex','twin','married','edu-left-hs','edu-hs','edu-sc','cig','first-born','alcohol','drugs','working','prenatal','ark','ein','har','mia','pen','tex','was']
        ihdp_data_compressed['y_factual'] = data['y_factual']
        ihdp_data_compressed['y_cfactual'] = data['y_cfactual']
        ihdp_data_compressed['ite'] = (ihdp_data_compressed['y_factual'] - ihdp_data_compressed['y_cfactual'])*(2*ihdp_data_compressed['treatment']-1)
        cols_list = ihdp_data_compressed.columns.tolist()
        ihdp_data_compressed = ihdp_data_compressed[cols_list[1:26] + cols_list[0:1] + cols_list[26:]]
        variable_dict = {'birth-weight':"1", # Anchor variable
                        'head-circumference':"2", # Kept
                        'pre-term': "3", # Kept
                        'birth-order': "4", # Kept
                        'neonatal':"5", # Kept
                        'age':"6", # Kept
                        'sex':"7", # Kept
                        'twin':"8",
                        'married':"9",
                        'edu-left-hs':"10",
                        'edu-hs':"10",
                        'edu-sc':"10",
                        'cig':"11",
                        'first-born':"12",
                        'alcohol':"13",
                        'drugs':"14", # Kept
                        'working':"15",
                        'prenatal':"16", # Kept
                        'ark':"17",
                        'ein':"17",
                        'har':"17",
                        'mia':"17",
                        'pen':"17",
                        'tex':"17",
                        'was':"17"}
        ihdp_data_compressed.columns = ["1","2","3","4","5","6","7","8","9","10","10","10","11","12","13","14","15","16","17","17","17","17","17","17","17","treatment", "y_factual", "y_cfactual","ite"]
        return ihdp_data_compressed, variable_dict, true_ate
    
if dataset == 'cattaneo':
	def get_cattaneo_compressed():
		cattaneo_ori = pd.read_stata('data/cattaneo2.dta')
		cattaneo_ori['mmarried'] = cattaneo_ori['mmarried'].replace('married', 1)
		cattaneo_ori['mmarried'] = cattaneo_ori['mmarried'].replace('notmarried', 0)
		cattaneo_ori['mbsmoke'] = cattaneo_ori['mbsmoke'].replace('smoker', 1)
		cattaneo_ori['mbsmoke'] = cattaneo_ori['mbsmoke'].replace('nonsmoker', 0)
		cattaneo_ori['fbaby'] = cattaneo_ori['fbaby'].replace('Yes', 1)
		cattaneo_ori['fbaby'] = cattaneo_ori['fbaby'].replace('No', 0)
		cattaneo_ori['prenatal1'] = cattaneo_ori['prenatal1'].replace('Yes', 1)
		cattaneo_ori['prenatal1'] = cattaneo_ori['prenatal1'].replace('No', 0)
		cattaneo_ori = cattaneo_ori.drop(['msmoke', 'lbweight'], axis=1)
		cols_to_norm = ['mage','medu','fage','fedu','nprenatal','monthslb','order','prenatal','birthmonth']
		cattaneo_norm = cattaneo_ori.copy()
		cattaneo_norm[cols_to_norm] = cattaneo_norm[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.std()))
		cols_list = cattaneo_norm.columns.to_list()
		cattaneo_norm = cattaneo_norm[cols_list[7:14] + cols_list[17:19] + cols_list[1:7] + cols_list[15:17] + cols_list[19:] + cols_list[14:15]+ cols_list[0:1]]
		cattaneo_norm.columns = cattaneo_norm.columns.to_list()[:-2] + ['treatment','y']
		variable_dict = {'mage':"1",
						'medu':"2",
						'fage': "3",
						'fedu': "4",
						'nprenatal':"5",
						'prenatal1':"6",
						'prenatal':"7",
						'monthslb':"8",
						'order':"9",
						'birthmonth':"10",
						'mmarried':"11",
						'mhisp':"12",
						'mrace':"13",
						'frace':"14",
						'fhisp':"15",
						'foreign':"16",
						'alcohol':"17",
						'deadkids':"18",
						'fbaby':"19"}
		cattaneo_compressed = cattaneo_norm.copy()
		cattaneo_compressed.columns = list(variable_dict.values()) + ['treatment','y']
		cat_columns = cattaneo_compressed.select_dtypes(['category']).columns
		cattaneo_compressed[cat_columns] = cattaneo_compressed[cat_columns].apply(lambda x: x.cat.codes)
		return cattaneo_compressed, variable_dict


In [4]:
if dataset == 'IHDP':
    data, variable_dict, true_ate = ihdp_data_prep()
    ite = data['ite']
    Y_og = data['y_factual']
    T_og = data['treatment']
    ite = data['ite']
    X_og = data.drop(columns = ['y_cfactual', 'y_factual', 'ite', variable_dict['twin'], variable_dict['married'],
                                                        variable_dict['edu-left-hs'] , variable_dict['edu-hs'],variable_dict['edu-sc'],variable_dict['cig'],
                                                        variable_dict['first-born'], variable_dict['alcohol'], variable_dict['working'],
                                                        variable_dict['ark'],variable_dict['ein'],variable_dict['har'],variable_dict['mia'],
                                                        variable_dict['pen'],variable_dict['tex'], variable_dict['was']])
    X, X_test, Y, y_test, T, t_test, ite_train, ite_test = train_test_split( X_og, Y_og, T_og, ite, test_size=0.2)
    
if dataset == 'jobs':
    data_path="data/sample_data.csv"
    dataset='jobs'
    data = pd.read_csv(data_path)
    data = data.rename(columns={"treat": "treatment"})
    Y_og = data['re78']
    T_og = data['treatment']
    X_og = data.drop(["treatment", "re78"], axis=1)
    X, X_test, Y, y_test, T, T_test = train_test_split( X_og, Y_og, T_og, test_size=0.2)
    true_att = 1676.3426
    
if dataset == 'cattaneo':
    data, variable_dict = get_cattaneo_compressed()
    Y_og = data['y']
    T_og = data['treatment']
    X_og = data.drop(columns = ['y', '5', '6', '7', '8', '9', '12', '15'], axis = 1)
    X, X_test, Y, y_test, T, t_test = train_test_split( X_og, Y_og, T_og, test_size=0.2)

In [5]:
if dataset == 'IHDP':
    T_te_rmse, S_te_rmse, X_te_rmse, DA_te_rmse, DR_te_rmse = [], [], [], [], []
    T_te_pehe, S_te_pehe, X_te_pehe, DA_te_pehe, DR_te_pehe = [], [], [], [], []

    for i in range(5):
        n = 100
        models = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=int(n/100))
        T_learner = TLearner(models=models)
        T_learner.fit(Y, T, X=X)
        T_te = T_learner.effect(X_test)
        T_te_rmse.append(np.abs(T_te - true_ate))
        T_te_pehe.append(np.sqrt(np.mean(np.square(T_te - ite_test))))

        S_learner = SLearner(overall_model=models)
        S_learner.fit(Y, T, X=X)
        S_te = S_learner.effect(X_test)
        S_te_rmse.append(np.abs(S_te - true_ate))
        S_te_pehe.append(np.sqrt(np.mean(np.square(S_te - ite_test))))

        propensity_model = RandomForestClassifier(n_estimators=100, max_depth=6, min_samples_leaf=int(n/100))
        X_learner = XLearner(models=models, propensity_model=propensity_model)
        X_learner.fit(Y, T, X=X)
        X_te = X_learner.effect(X_test)
        X_te_rmse.append(np.abs(X_te - true_ate))
        X_te_pehe.append(np.sqrt(np.mean(np.square(X_te - ite_test))))

        DA_learner = DomainAdaptationLearner(models=models, final_models=models, propensity_model=propensity_model)
        DA_learner.fit(Y, T, X=X)
        DA_te = DA_learner.effect(X_test)
        DA_te_rmse.append(np.abs(DA_te - true_ate))
        DA_te_pehe.append(np.sqrt(np.mean(np.square(DA_te - ite_test))))

        DR_learner = DRLearner(model_regression=models, model_propensity=propensity_model, model_final=models, cv=5)
        DR_learner.fit(Y, T, X=X)
        DR_te = DR_learner.effect(X_test)
        DR_te_rmse.append(np.abs(DR_te - true_ate))
        DR_te_pehe.append(np.sqrt(np.mean(np.square(DR_te - ite_test))))

print()
print("T-Learner sqrt(ePEHE): ", "{:.2f}".format(np.mean(T_te_pehe)), ' \pm ', "{:.2f}".format(np.std(T_te_pehe)))
print("T-Learner eATE: ", "{:.2f}".format(np.mean(T_te_rmse)), ' \pm ', "{:.2f}".format(np.std(T_te_rmse)))
print()
print("S-Learner sqrt(ePEHE): ", "{:.2f}".format(np.mean(S_te_pehe)), ' \pm ', "{:.2f}".format(np.std(S_te_pehe)))
print("S-Learner eATE: ", "{:.2f}".format(np.mean(S_te_rmse)), ' \pm ', "{:.2f}".format(np.std(S_te_rmse)))
print()
print("X-Learner sqrt(ePEHE): ", "{:.2f}".format(np.mean(X_te_pehe)), ' \pm ', "{:.2f}".format(np.std(X_te_pehe)))
print("X-Learner eATE: ", "{:.2f}".format(np.mean(X_te_rmse)), ' \pm ', "{:.2f}".format(np.std(X_te_rmse)))
print()
print("DA-Learner sqrt(ePEHE): ", "{:.2f}".format(np.mean(DA_te_pehe)), ' \pm ', "{:.2f}".format(np.std(DA_te_pehe)))
print("DA-Learner eATE: ", "{:.2f}".format(np.mean(DA_te_rmse)), ' \pm ', "{:.2f}".format(np.std(DA_te_rmse)))
print()
print("DR-Learner sqrt(ePEHE): ", "{:.2f}".format(np.mean(DR_te_pehe)), ' \pm ', "{:.2f}".format(np.std(DR_te_pehe)))
print("DR-Learner eATE: ", "{:.2f}".format(np.mean(DR_te_rmse)), ' \pm ', "{:.2f}".format(np.std(DR_te_rmse)))


T-Learner sqrt(ePEHE):  1.87  \pm  0.01
T-Learner eATE:  0.92  \pm  0.78

S-Learner sqrt(ePEHE):  2.28  \pm  0.34
S-Learner eATE:  1.51  \pm  0.82

X-Learner sqrt(ePEHE):  1.67  \pm  0.01
X-Learner eATE:  0.76  \pm  0.65

DA-Learner sqrt(ePEHE):  1.77  \pm  0.03
DA-Learner eATE:  0.88  \pm  0.74

DR-Learner sqrt(ePEHE):  2.74  \pm  0.15
DR-Learner eATE:  2.12  \pm  0.83


In [6]:
def policy_risk(T,Y, mu_0_pred, mu_1_pred):
    y_0_true = Y[T==0].tolist()
    y_1_true = Y[T==1].tolist()
    n = len(y_0_true)
    d_opt = (y_1_true > y_0_true)
    d_hat = (mu_1_pred > mu_0_pred)
    policy_risk_value = np.mean(np.abs((mu_1_pred - mu_0_pred) * (2 * (d_hat != d_opt) - 1)))
    return policy_risk_value

if dataset == 'jobs':
    T_te_rmse, S_te_rmse, X_te_rmse, DA_te_rmse, DR_te_rmse = [], [], [], [], []
    # T_te_pr, S_te_pr, X_te_pr, DA_te_pr, DR_te_pr = [], [], [], [], []

    for i in range(5):
        n = 100
        # print(X.shape, T.shape, Y.shape)
        models = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=int(n/100))
        T_learner = TLearner(models=models)
        T_learner.fit(Y, T, X=X)
        T_te = T_learner.effect(X_test)
        # mu_0_pred = T_learner.models[1].predict(X)
        # mu_1_pred = T_learner.models[0].predict(X)
        T_te_rmse.append(np.abs(T_te - true_att))
        # T_te_pr.append(policy_risk(T,Y,mu_0_pred, mu_1_pred))

        S_learner = SLearner(overall_model=models)
        S_learner.fit(Y, T, X=X)
        S_te = S_learner.effect(X_test)
        # X_prime = X
        # X_prime['treatment'] = np.zeros_like(T)
        # mu_0_pred = S_learner.overall_model.predict(X_prime)
        # X_prime['treatment'] = np.ones_like(T)
        # mu_1_pred = S_learner.overall_model.predict(X_prime)
        S_te_rmse.append(np.abs(S_te - true_att))
        # S_te_pr.append(policy_risk(T,Y,mu_0_pred, mu_1_pred))

        propensity_model = RandomForestClassifier(n_estimators=100, max_depth=6, min_samples_leaf=int(n/100))
        X_learner = XLearner(models=models, propensity_model= propensity_model)
        X_learner.fit(Y, T, X=X)
        X_te = X_learner.effect(X_test)
        # mu_0_pred = X_learner.models[1].predict(X)  # Predictions for T=1
        # mu_1_pred = X_learner.models[0].predict(X)    # Predictions for T=0
        X_te_rmse.append(np.abs(X_te - true_att))
        # X_te_pr.append(policy_risk(T,Y,mu_0_pred, mu_1_pred))

        DA_learner = DomainAdaptationLearner(models=models, final_models=models, propensity_model=propensity_model)
        DA_learner.fit(Y, T, X=X)
        DA_te = DA_learner.effect(X_test)
        # mu_0_pred = DA_learner.models[1].predict(X)  # Predictions for T=1
        # mu_1_pred = DA_learner.models[0].predict(X)    # Predictions for T=0
        DA_te_rmse.append(np.abs(DA_te - true_att))
        # DA_te_pr.append(policy_risk(T,Y,mu_0_pred, mu_1_pred))

        DR_learner = DRLearner(model_regression=models, model_propensity=propensity_model, model_final=models, cv=5)
        DR_learner.fit(Y, T, X=X)
        DR_te = DR_learner.effect(X_test)
        # mu_0_pred = DR_learner.models[1].predict(X)  # Predictions for T=1
        # mu_1_pred = DR_learner.models[0].predict(X)    # Predictions for T=0
        DR_te_rmse.append(np.abs(DR_te - true_att))
        # DR_te_pr.append(policy_risk(T,Y,mu_0_pred, mu_1_pred))

print()
# print("T-Learner eRP: ", "{:.2f}".format(np.mean(T_te_pr)), ' \pm ', "{:.2f}".format(np.std(T_te_pr)))
print("T-Learner eATT: ", "{:.2f}".format(np.mean(T_te_rmse)), ' \pm ', "{:.2f}".format(np.std(T_te_rmse)))
print()
# print("S-Learner eRP: ", "{:.2f}".format(np.mean(S_te_pr)), ' \pm ', "{:.2f}".format(np.std(S_te_pr)))
print("S-Learner eATT: ", "{:.2f}".format(np.mean(S_te_rmse)), ' \pm ', "{:.2f}".format(np.std(S_te_rmse)))
print()
# print("X-Learner eRP: ", "{:.2f}".format(np.mean(X_te_pr)), ' \pm ', "{:.2f}".format(np.std(X_te_pr)))
print("X-Learner eATT: ", "{:.2f}".format(np.mean(X_te_rmse)), ' \pm ', "{:.2f}".format(np.std(X_te_rmse)))
print()
# print("DA-Learner eRP: ", "{:.2f}".format(np.mean(DA_te_pr)), ' \pm ', "{:.2f}".format(np.std(DA_te_pr)))
print("DA-Learner eATT: ", "{:.2f}".format(np.mean(DA_te_rmse)), ' \pm ', "{:.2f}".format(np.std(DA_te_rmse)))
print()
# print("DR-Learner eRP: ", "{:.2f}".format(np.mean(DR_te_pr)), ' \pm ', "{:.2f}".format(np.std(DR_te_pr)))
print("DR-Learner eATT: ", "{:.2f}".format(np.mean(DR_te_rmse)), ' \pm ', "{:.2f}".format(np.std(DR_te_rmse)))


T-Learner eATT:  0.92  \pm  0.78

S-Learner eATT:  1.51  \pm  0.82

X-Learner eATT:  0.76  \pm  0.65

DA-Learner eATT:  0.88  \pm  0.74

DR-Learner eATT:  2.12  \pm  0.83


In [7]:
if dataset == 'cattaneo':
    T_te_rmse, S_te_rmse, X_te_rmse, DA_te_rmse, DR_te_rmse = [], [], [], [], []

    for i in range(5):
        n = 100
        models = GradientBoostingRegressor(n_estimators=100, max_depth=6, min_samples_leaf=int(n/100))
        T_learner = TLearner(models=models)
        T_learner.fit(Y, T, X=X)
        T_te = T_learner.effect(X_test)
        T_te_rmse.append(np.abs(T_te))

        S_learner = SLearner(overall_model=models)
        S_learner.fit(Y, T, X=X)
        S_te = S_learner.effect(X_test)
        S_te_rmse.append(np.abs(S_te))

        propensity_model = RandomForestClassifier(n_estimators=100, max_depth=6, min_samples_leaf=int(n/100))
        X_learner = XLearner(models=models, propensity_model= propensity_model)
        X_learner.fit(Y, T, X=X)
        X_te = X_learner.effect(X_test)
        X_te_rmse.append(np.abs(X_te))

        DA_learner = DomainAdaptationLearner(models=models, final_models=models, propensity_model=propensity_model)
        DA_learner.fit(Y, T, X=X)
        DA_te = DA_learner.effect(X_test)
        DA_te_rmse.append(np.abs(DA_te - true_att))

        DR_learner = DRLearner(model_regression=models, model_propensity=propensity_model, model_final=models, cv=5)
        DR_learner.fit(Y, T, X=X)
        DR_te = DR_learner.effect(X_test)
        DR_te_rmse.append(np.abs(DR_te))

print()
print("T-Learner ATE: ", "{:.2f}".format(np.mean(T_te_rmse)), ' \pm ', "{:.2f}".format(np.std(T_te_rmse)))
print()
print("S-Learner ATE: ", "{:.2f}".format(np.mean(S_te_rmse)), ' \pm ', "{:.2f}".format(np.std(S_te_rmse)))
print()
print("X-Learner ATE: ", "{:.2f}".format(np.mean(X_te_rmse)), ' \pm ', "{:.2f}".format(np.std(X_te_rmse)))
print()
print("DA-Learner ATE: ", "{:.2f}".format(np.mean(DA_te_rmse)), ' \pm ', "{:.2f}".format(np.std(DA_te_rmse)))
print()
print("DR-Learner ATE: ", "{:.2f}".format(np.mean(DR_te_rmse)), ' \pm ', "{:.2f}".format(np.std(DR_te_rmse)))


T-Learner ATE:  0.92  \pm  0.78

S-Learner ATE:  1.51  \pm  0.82

X-Learner ATE:  0.76  \pm  0.65

DA-Learner ATE:  0.88  \pm  0.74

DR-Learner ATE:  2.12  \pm  0.83
